# Recommend Lines

Full pipeline of lines recommendation for home team.

In [1]:
%load_ext autoreload

import sys
sys.path.append("/Users/luisd/dev/NHL_stats")
print(sys.path)

['', '/Users/luisd/sw/anaconda3/envs/nhl_stats/lib/python36.zip', '/Users/luisd/sw/anaconda3/envs/nhl_stats/lib/python3.6', '/Users/luisd/sw/anaconda3/envs/nhl_stats/lib/python3.6/lib-dynload', '/Users/luisd/.local/lib/python3.6/site-packages', '/Users/luisd/sw/anaconda3/envs/nhl_stats/lib/python3.6/site-packages', '/Users/luisd/sw/anaconda3/envs/nhl_stats/lib/python3.6/site-packages/IPython/extensions', '/Users/luisd/.ipython', '/Users/luisd/dev/NHL_stats']


In [4]:
import pickle
from os import path

import numpy as np

from ReinforcementLearning.NHL.playbyplay.playbyplay_data import Season
from ReinforcementLearning.NHL.player.player_type import PlayerType
from ReinforcementLearning.NHL.lines.category import CategoryFetcher
from ReinforcementLearning.NHL.lines.valuation import QValuesFetcherFromDict, QValuesFetcherFromGameData
from ReinforcementLearning.NHL.lines.recommender import LineRecommender


In [7]:
from ReinforcementLearning.NHL.playbyplay.state_space_data import HockeySS
"""Initialization"""
db_root = '/Users/luisd/dev/NHL_stats/data'
repoCode = '/Users/luisd/dev/NHL_stats'
# Montreal received Ottawa on march 13, 2013, let's convert game date to game code
gameId = Season.get_game_id(db_root=db_root, home_team_abbr='MTL', date_as_str='2013-03-13')

repoModel = path.join(repoCode,
                           'ReinforcementLearning/NHL/playerstats/offVSdef/Automatic_classification/MODEL_perceptron_1layer_10units_relu')

# Now lets get game data
season = Season(db_root=db_root, year_begin=2012, repo_model=repoModel)  # Season.from_year_begin(2012) # '20122013'
mtlott = season.pick_game(gameId)

# players_classes = mtlott.pull_players_classes_from_repo_address(True, True, 20, repoModel, number_of_games=30)

# prediction of the lines that the 'away' team will use:
df, away_lines = mtlott.get_away_lines(accept_repeated=True)

# === Now we get the indices in the Q-values tables corresponding to lines

# Line translation table
linedict  = HockeySS(db_root)
linedict.make_line_dictionary()
linedict  = linedict.line_dictionary

# Get lines and translate them
playersCode  =   mtlott.encode_line_players()
linesCode    =   np.array( [[mtlott.recode_line(linedict, a) for a in b] for b in playersCode] )


# Load the Qvalues table
Qvalues = \
pickle.load(open(path.join(repoCode, 'ReinforcementLearning/NHL/playbyplay/data/stable/RL_action_values.p'), 'rb'))[
    'action_values']

# Visualize it dimensions (period x differential x away line's code x home line's code)
print('Q-table dimensions: ', Qvalues.shape)

# Get the Q-value for that specific line
iShift = 0  # First shift
lineShifts = mtlott.lineShifts.as_df(team='both', equal_strength=mtlott.shifts_equal_strength,
                                   regular_time=mtlott.shifts_regular_time, min_duration=20)

player_classes = mtlott.players_classes_mgr.get(equal_strength=True, regular_time=True, min_duration=20, nGames=30) # TODO: why these parameters?
plList = list(player_classes.loc[lineShifts['playersID'].iloc[iShift][0]]['firstlast'].values) + \
         list(player_classes.loc[lineShifts['playersID'].iloc[iShift][1]]['firstlast'].values)
diff = mtlott.recode_differential(lineShifts.iloc[iShift].differential)
period = mtlott.recode_period(lineShifts.iloc[iShift].period)
q_values = Qvalues[period, diff, linesCode[iShift, 0], linesCode[iShift, 1]]
print('[diff = %d, period = %d] First shift: \n\thome team: %s, %s, %s \n\taway team: %s, %s, %s \n\tQvalue: %.2f' % (
diff, period, plList[0], plList[1], plList[2], plList[3], plList[4], plList[5], q_values))



q_value_tuples = [
    ([PlayerType.NEUTRAL, PlayerType.NEUTRAL, PlayerType.OFFENSIVE],
     [PlayerType.OFFENSIVE, PlayerType.DEFENSIVE, PlayerType.DEFENSIVE],
    25),
    ([PlayerType.NEUTRAL, PlayerType.NEUTRAL, PlayerType.OFFENSIVE],
     [PlayerType.NEUTRAL, PlayerType.NEUTRAL, PlayerType.NEUTRAL],
    12),
    ([PlayerType.NEUTRAL, PlayerType.NEUTRAL, PlayerType.NEUTRAL],
     [PlayerType.OFFENSIVE, PlayerType.DEFENSIVE, PlayerType.DEFENSIVE],
    20),
    ([PlayerType.NEUTRAL, PlayerType.NEUTRAL, PlayerType.NEUTRAL],
     [PlayerType.NEUTRAL, PlayerType.NEUTRAL, PlayerType.NEUTRAL],
    1),
]

q_values_fetcher_from_game_data = QValuesFetcherFromGameData(game_data=mtlott, lines_dict=linedict, q_values=Qvalues)
q_values_fetcher_from_tuples = QValuesFetcherFromDict.from_tuples(q_value_tuples)

line_rec = LineRecommender(
    player_category_fetcher=CategoryFetcher(data_for_game=mtlott),
    q_values_fetcher=q_values_fetcher_from_game_data) # q_values_fetcher_from_tuples)

home_lines_rec = line_rec.recommend_lines_maximize_average(
                                home_team_players_ids=mtlott.get_home_players(repoModel),
                                away_team_lines = away_lines, examine_max_first_lines=None)
print(home_lines_rec)

# let's translate these numbers into names:
# input is ~ [(656, 27, 31), (1380, 389, 1035), (8, 9, 1164), (281, 13, 14)]
line_no = 1
for a_line in home_lines_rec:
    first_guy, second_guy, third_guy = a_line
    print("Line %d: %s, %s, %s" % (
        line_no,
        mtlott.rf_wc[mtlott.rf_wc['player.id'] == first_guy]['numfirstlast'].tolist()[0],
        mtlott.rf_wc[mtlott.rf_wc['player.id'] == second_guy]['numfirstlast'].tolist()[0],
        mtlott.rf_wc[mtlott.rf_wc['player.id'] == third_guy]['numfirstlast'].tolist()[0],
    ))
    line_no += 1


/Users/luisd/dev/NHL_stats/ReinforcementLearning/NHL/playbyplay/playbyplay_data.py:54: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  gameInfo    =   gameInfo[gameInfo['gameDate']==date_as_str][gameInfo['teamAbbrev']==home_team_abbr]


INFO:tensorflow:Restoring parameters from /Users/luisd/dev/NHL_stats/ReinforcementLearning/NHL/playerstats/offVSdef/Automatic_classification/MODEL_perceptron_1layer_10units_relu/MODEL_perceptron_1layer_10units_relu
Q-table dimensions:  (3, 5, 10, 10)
[diff = 2, period = 0] First shift: 
	home team: DAVID DESHARNAIS, BRENDAN GALLAGHER, MAX PACIORETTY 
	away team: KYLE TURRIS, JAKOB SILFVERBERG, COLIN GREENING 
	Qvalue: 3.17
Q value dictionary has 16 entries
Best fitness: 50.33 by formation [(1380, 389, 8), (9, 1035, 1164), (13, 14, 656), (281, 27, 31)]
Best fitness: 51.67 by formation [(1380, 389, 8), (9, 1035, 1164), (13, 14, 281), (656, 27, 31)]
=======> Took 0.29 secs. to look at 1 first-lines; I think we have around 62.52 secs. to go
Best fitness: 51.67 by formation [(656, 27, 31), (1380, 389, 1035), (8, 9, 1164), (281, 13, 14)]
=======> Took 58.12 secs. to look at 219 first-lines; I think we have around 0.27 secs. to go
ALL DONE!!!!!!
Best fitness: 51.67 by formation 
[(656, 27, 31

In [6]:
# do_it_together()